In [3]:
#so where to go from here
#It has some evidence that it trains, so I shouldnt abandon it by any means
#it is based off of one of the most reputable and impressive AI algos
#so I believe it's worth keeping


#there are some interesting directions / things we need to fix:

#Skips in the most dont work, and in general the flow of the algorithm doesnt give enough freedom
#to the architecture design

#i.e. we really need to allow arbitrary flow

#we can copy what ENAS does and go through each node an pick a previous node,
#basically it's a prior that we use all of the nodes available

#ideally we would like an upper bound on what our computer can handle, and the network
#picks the best option based on a trade off of performance/complexity or whatever

#the parameters scaling kind of does that, although for groups for example it reduces the parameters
#but increases computation time (and I think memory, not sure)

In [4]:
#so basically we need to restructure the design so it has more freedom

In [5]:
#Honestly I just want to get it to work and have a lot of freedom, then hopefully turn it on itself
#for example, making a better controller, or better softmaxs

#****
#I would like to decouple embedding size from LSTM size
#I think it may be desirable to bottleneck the embedding, but give the LSTM more capacity

#so basically we would have an "adapter" layer that changes the embedding size to fit into the LSTM

#does that make sense?

#I think so... although if the LSTM is smaller I guess it doesnt make sense right?

#if the LSTM is smaller than the embedding, it doesnt make sense because we are already bottlenecking the
#ebmedding with the LSTM

#if the embedding is smaller than the lstm, then this will make it so we arent forced to use a larger embedding
#size than we want, which may force embeddings to become more similar, which is what we want

In [6]:
#So one of the key thing we need to look at is how to give the execution flow maximum power

In [7]:
#Basically, we want to give as much control as possible, and really open up neural networks
#to be dynamic graphs, and have tree structures, etc

#for that we need conditionals, branching, loops, maybe jumps, etc

In [8]:
# 1. A form of conditional repetition or conditional jump (while, for, if and goto)
# 2. A way to read and write to some storage mechanism (variables)

In [9]:
#So basically we want to be able to allow the network to dynamically create new nodes, put them together, etc

In [10]:
#idea: what if we use the balanced examples idea to make it so that we can have sparse rewards

In [11]:
#Well of course the obvious thing is that we need to get the system working before we can justify many
#further changes

#we can give the system all the expressivity in the world, but if it cant train its all moot

#the issue is I'm not sure how to do it, and I honestly dont know if I have hyperparameters slightly wrong,
#or if this is impossible to do on CPU (decent probability), or what


#I think for now I can probably try to get the spec to what ENAS had, i.e. we go through each of the nodes,
#and we specify what the previous node would be (we would add one choice for this) and the activation is already
#chosen

#so it really would be as simple as that, what would the previous node be, and possible, what kind of interaction?
#such as, is it a concat then bottleneck conv, add, average, mult, etc

#and then we can also add the trick where we average the nodes that we're outputs to anything else

In [12]:
#well if you think about it right now we are just defining a basic feed forward, and it is using a kind of 
#attention skip mechanism (its probably bad)

#what we really want to do is allow complex branching, recombing, etc

#like we really want the traditional LSTM to be reachable for example

#a good place to start is making something reachable

In [14]:
#So lets start by making a couple things I care about reachable:

#First step would be forget gate only LSTM

#For that we need matrix multiplies, i.e. linear layers, right?

#can you represent a LSTM with linear layers?
#I think so, a Linear is just a matrix
#so in theory if you have linears you can have an LSTM
#and since Yann LeCunn said that a Conv 1x1, just Convs should work too.

# h = Linear(h-1, )

In [15]:
#so lets see

#f_t = sigmoid(conv2d1x1(ht-1) + conv2d1x1(x_t))
#c_t = f_t * c_t-1 + (1 - f_t) * tanh(conv...(ht-1) + conv(x_t))

#so what is needed for that. For example we pick a previous node and an activation
#so assuming ht-1 and x are nodes, we could do conv2d(x) -> (identity) -> + -> conv2d(h_t) -> (sigmoid)
#issue: we face severe limitations, or overcomplication by doing this

In [16]:
#we really

In [17]:
#Okay well Ineed to think for a minute and plan my trajectory
#One: I have a lot of work put into this AlphaZeroEnas thing.
#Two: If it works, I will get a lot of fame, i.e. its useful and people want it.
#Three: It is related to and therefore helps me learn about program synthesis, which is the ultimate goal

#Out of all of the things I could practically think of for a self feedback loop (i.e. increasing my own
#productivity), automatically creating optimized architectures (and hopefully hyperparameters) would be huge

#The other one would be maybe debugging, but that is a much harder issue to tackle

#Plus, the area is pretty well researched.

#Honestly, I should improve AlphaZeroEnas as much as I can, maybe put it on the backburner and work on Yu's project
#and then in the future when I have a good GPU I can refine it and write a paper.

#For now I want to make sure the expressivity is to a point that I'm okay with though.

In [18]:
#So yeah what kind of expressivity do I need to allow for what I want

In [19]:
#Lets do it piece by piece:
#LSTM:

#I need to access a previous output, and it may be convenient to extrapolate to having access to all previous outputs
#downside: potentially high memory usage, and if we were going to allow attention over all of those for example,
#it may create easy pitfalls for the system to get tripped up on

#But at the very least, we want to all it to have access to the last output

In [20]:
#So I think I need to fundamentally redesign how I think about the flow of the program

#It really should start looking more like a turing machine

#1. A form of conditional repetition or conditional jump (while, for, if and goto)
# 2. A way to read and write to some storage mechanism (variables)


In [21]:
#So that means, the network needs to be able to write to and read from a persistent memory (matrix)
#For that we can allow the network to create...

In [22]:
#Well this gets to another interesting point.
#Do we want to allow the system to be dynamic, or to lock it into a certain thing

In [23]:
#Maybe we can take the approach of pointer nets and dynamically expand as needed

In [24]:
#So for example this would come back to the example I thought of for the GAN MCTS for transitions rather than
#policies

In [25]:
#We have a node which can choose to expand, or point to a previous choice

In [26]:
#We can kind of view the previous choices as variables

In [27]:
#Maybe a marrying of program synthesis ideas and architecture generation can result in something better than
#the two.

#I think I need more knowledge about the potential power of these types of algorithms

#But for now lets try to work through it

In [28]:
#So basically the goal is that we want a type of turing complete, maximum from basics neural net

In [29]:
#To make it so that it doesnt fail due to shape mismatches and such, we should lock the system into a certain
#configuration once it starts, unless there is an easy work around, such as padding, using 1x1 conv to combine,
#etc

In [30]:
# 1. A form of conditional repetition or conditional jump (while, for, if and goto)
# 2. A way to read and write to some storage mechanism (variables)

#so for example, we want to allow the system to continue until it reaches a certain number of iterations
#or a certain size (certian upper bounds that we limit it to)

#we want the system to be dynamic so that it can can choose to create anything as in needs

#but the question is, if we're using upper bounds anyways, it could be static, and then system merely will
#choose what portion of the network we are utilizing

In [31]:
#One thing I always thought was interesting was trying to emulate the brain structure

#I.e. imagine all of the neurons are one giant matrix, and the system determines how they are connected together

#i.e. it uses a set of operations over those numbers to accomplish some task

In [32]:
#so for example lets say we give it access to all of the basic pytorch tensor actions, such as 
#matrix multiply and elementwise operations (there are certainly more but this is okay for now)

#Then the system basically samples a subset of the nodes. We want a certain level of sparsity
#so for example, lets say we want 1% sparsity for a brain of size 100x100

#so we have 10000 numbers in total, and we would only pick on average 100 for a given operation

#so basically we have a big matrix of numbers, i.e. some Parameter(tensor(100, 100))

In [33]:
#Then we randomly sample a mask over it...
#Issue: Outputing that size of a matrix (if we make the amount of numbers much larger) isn't possible

In [34]:
#we have to go back to the dynamics of the brain. it appears to act locally, and maybe there is a "controller"
#that surpresses or encourages certain areas

#We could kind of view that as attention

#So the "consciousness" is attending to processes that are important at a given time

In [35]:
#But at the end of the day, its a bunch of neurons that are connected sparsely, and they interact to
#do certain functions.

In [36]:
#So for example what if we got a list of neurons [5, 3, 6] and assigned them to variable 1
#and we got another list of neurons [9, 1, 2] and assigned them to variable 2
#then we matrix multiplied them (it automatically reshapes if possible)

#and we compare that result to the real output, perhaps we have a fixed 
#"head" that we refer back to, and constrains the output to a certain size

In [37]:
#So the idea is basically rather than having different layers that we fit together, we just have one big
#matrix, that we sample from to perform certain actions

In [38]:
#And the idea I have basically is that the controller picks a bunch of paths through the neurons,
#i.e. we do some batch UCT select, or many different processes each picking a neuron, then we group those together
#and they get combined at the end

In [39]:
#but we need interactions from many different neurons
#So yeah in theory we want to do...

In [40]:
#So lets think for a bit. What do we want
#We want to test on something, such as cifar10
#for that we will need to require that the input is (batch_size, 3, 32, 32) and that the output is 10
#So basically I think we would have a stem which matches that size, and then flattens,
#and we would have a output which goes in and takes that same flattened size

In [41]:
#That would put a requirement that all of the operation inside used that same "flattened" number of neurons
#Which is less than idea, but coudl work. Also we can bottleneck the flattened number to make it smaller

In [42]:
#So basically what we're thinkg about is the middle part, i.e. we have one big matrix of neurons, and we 
#random select some to interact with others. i.e. any differentiable operation between them, and perhaps for now
#recurrence is prevented, we want it though so we can consider it.

In [43]:
#so the idea is that the two neurons sampled will be the same size

In [44]:
#and basically we need to allow them to interact in different ways
#so they can add together, multiply, matrix multiply, etc

In [45]:
#Well we have to consider theres a difference between weights and neurons

In [46]:
#But they are represented in their weights, no?
#the "brain" is a series of weights

#so basically we are passing along an input from a sensor (image data)
#and we are transforming it with matrix operations

In [47]:
#idea: what if we applied alphazero to determine the behavior of a single neuron
#and then we replicate that behavior between many and do a random search (or alpha zero)?

In [48]:
#so for example lets say the alpha zero determines a bias and an initialization for a "neuron"
#how would it even do that lol. it's sequential. I can do a GAN alpha zero maybe, but that isn't quite the same
#it isn't using UCT

In [49]:
#Let me step back for a minute and think about what I'd like to research and what I would ideally like to happen

#Step one: make an efficient architecture search and at least use it to simplify future research

#Step two: look into meta learning so that we dont have to retrain from scratch every time, and we can have more
#personalization, rather than averagin among everyone

#STep three: We want to overcome backprop. It probably isnt what the brain uses, and limits us to a small
#subset of functions, because it has to be differentiable. We really want more of a "chaotic" disconnected
#approach. The issue with sequential planning is that it becomes like exponentially harder to predict.
#and it also is very costly

#it is essential in some aspects, but the brain isn't coordinating everything, intelligence emerges from
#individual interactions. well that said we dont really know. We are only aware of what our consciousness is
#aware of, and there are plenty of processes that happen without our control. So really DNA has set a framework
#and the brain is interacting with those components. There generalization between the two is that they both adapt
#to meet the requirements of their environment, i.e. DNA that is more fit will replicate, brains that are more
#fit will replicate. so where are we going with this. At the end of the day, we dont have DNA, we need something
#that looks at a value, and tries to get that value perfect, for example a GAN does that, it sees what makes something
#good or bad, then it tries to get the generator to make the bad better so that the discrim cant tell the difference

#The key insight of Alpha Zero is focusing on a subset of the possible domain which is promising,
#i.e. we are biased towards looking at future with high value, and following the policy we imagined before
#and it explores in a theoretically principled way using UCT

In [50]:
#The question for example is how could we overcome backprop
#Basically we would need to create a set of mathematical operations such that given a set of operations,
#we change the parameters involved so that they will solve the problem

In [51]:
#So for example maybe we could have a network that does that. If it remembers well enough
#It can go through a set of operations, track history from each of them, and then learn an update to the
#parameters with a backward pass

In [52]:
#So basically the idea would be to learn a function approximation of a better function approximation

In [53]:
#And you probably could train that with a GAN or something, i.e. guess the performance of the network after the update
#then have the generator create parameters such that they are close to perfect performance

In [54]:
#Is there a way to do it asychronously?
#Well my understanding with the brain

In [55]:
#Well what's clear with the brain, unless we are very wrong on the theory,
#is that everything happens asychnronously.
#i.e. we dont wait for one thing to happen before the next, all neurons are just firing whenever

In [56]:
#My understanding is that they arent clear if it's a message encoding, based on the order of the firing,
#or if it's frequency based, based on the how quickly a connection is firing, it seems like it would more be
#based on the magnitude of the output, but probably they found that all of them fire with the same magnitude

In [57]:
#frequency vs message

In [58]:
#Message would basically mean that the brain develops patterns such that when certain messages are come accross
#A certain chain reaction is taking placing

In [59]:
#Personally it seems like it would be a lot harder to balance the message passing
#because one or two switched orders could totally mess up the message

In [60]:
#The frequency thing makes more sense to me
#And probably there's a different bias to each connection maybe?
#so like each connection from the different neurons have different indicators, like for example,
#maybe a certain connection indicates fear

#Like what if we had a turing complete machine that had a set of primitives

In [61]:
#So the inputs that take all of the neurons in learn different things from the different neurons
#so for example, its the degree of something, such as the degree of fear,
#and each connection is a different emotion or what have you

In [62]:
#Well if deep learning has showed anything (which is it the best approximation of the brain we have)
#its that the brain adapts over time to meet it's needs, and there are preprogrammed needs from DNA
#built in, i.e. the eyes

In [63]:
#so basically we have a system that takes in input, has intrinsic goals (which were specified by their environment
#through survival of the fittest)
#so things that are advantageous will be passed on 
#so the precise subgoals and submethods will be adapted to the task at hand

#but again we get to how would we replace backprop
#basically the idea is build a backprop replacement net
#but if we do that we want to make it more like the brain and ideally non differentiable

In [65]:
#and ideally what we really want is to have a differentiable net make a non differentiable one that is better,
#and then recursively call that until we have AGI, or at least a pretty strong non differentiable training algo

In [66]:
#So yeah I'm getting super ahead of myself
#What do I really want: Less f'ing around with arch designs and less hyperparameter tweaking
#I think alpha zero combined with ENAS can fix both of those

In [67]:
#So there are two ideas I want to consider:
#1. Having the arch designer make a new arch designer and use that new arch designers averaged scores as 
#the reward signal for the first arch designer

#2. Converting the systems own hyperparameters to being chosen by the system itself
#So for example with number of sims, we would predict a prior over how many sims we want to do
#and we would predict a value for what score we think we will get with that many sims
#and we would want to scale the score by how many sims we have
#so for example, maybe we do something similar to the parameter scaling, where we scale the num sims between
#0 and 1, and then scale it to whatever weight we want, for example 10%
#then if the for example 10 extra sims out of 10 max sims doesnt benefit....

# 100% would mean each sim needs to generate at least a 1% improvement for 100 sims

In [68]:
#Idk I can think about it and think about the weights, but yeah

In [69]:
#What else, the learning rate can be handled by fastai
#the batchsize is related to learning rate so that can be fastai
#number of memories maybe? so we can predict the memory size, predict the score based on that, etc.
#some other ones could be num_train_iters
#but for that, again probably more train iters would mean a higher score, so probably again we want a 
#divisor based on that

#maybe we could also predict a balance between 

In [70]:
#Some hyperparameters we may want to have the system predict:

#Num sims, and divide score by number of sims to prevent always max
#max_memories
#num_train_iters, would need to divide to prevent always max
#arch_train_batch_size, arch_train_learning_rate, possibly momentum
#it would be nice to predict the things for a cylical LR, maybe would allow super convergence

In [71]:
#Having the system predict it's own learning rate and such would also be very interesting

In [72]:
#So basically the idea is have the system predict all it's and the arch's hyperparmeters
#One issue is that if the controller blows (like at the start), the learning rate being f'd up
#will mess things up a lot

#We could maybe have a hand tweaked controller, then have it take over?
#or maybe we can just let it jostle around and gradient clip it.
#I think that might be fine
#we want it to be a bit chaotic, then settle into being a flow

In [73]:
#So anyways where did we get with all of this thinking
#basically: I hate hyperparameter tuning and arch design, and I think it's a waste of time
#I would like to make the system do all of it.
#So I would like to for one,

In [74]:
#Well lets see, I wanted to think about the idea of having a proxy controller, i.e. a controller that designs
#a controller that designs an architecture 

In [75]:
#We ideally want the two to have the same interface for designs architectures so that we can interchange them
#the benefit is that for certain things, such as choosing the number of layers to use, we can 
#have the other controller determine that, and then when it's not applicable we skip it

#so it basically we open up another level of control over the hyperparameter choinces. oh yeah, and
#of course the LSTM design, softmax design, embedding design, etc,

In [76]:
#Now the obvious question is that the algorithm I have right now is no where near robust enough to change everything
#we at the very least would need to...

In [77]:
#Well honestly one thing Ive noticed is that I'm just making a system to make sequential decisions! 
#It doesnt really matter if its for an architecture or whatever

In [78]:
#So yeah I need to consider about the end goal here:
#The end goal is not just to make an architecture designer, its to make one that makes better ones!

In [79]:
#So we have the insight that we can choose any decision where we get a signal back
#And basically anything that will influence the final score can be trained
#so for example, all of the arch hyperparameters will effect the score,
#and then all of the controller hyperparameters will through causality effect the arch

In [80]:
#for decisions where more is almost always better, we need a system where we balance performance/resources

In [81]:
#It would be nice to have a principled way to do this
#so for example, for number of sims, a reasonable number of sims might be between 10-100
#and basically we could set the max to be 100, and scale the 0-1 based on the max
#so for example, maybe it = 0-1 /max
#so the larger max is, the smaller the scaler is
#that intuitively kind of makes sense, where if we can do 3 sims, the effect per sim better be a lot bigger rather
#than if we are doing 1000 sims

In [82]:
#and probably we could do something like log(max) or log(log(max))
#or perhaps for example, while divisor >= 1: log(divisor)

In [138]:
import math
1/math.log(10000, 1.1) #SO I think this is saying basically we would need a 2.1% increase per unit to justify it
#which seems fairly reasonable
#for this it would 
#so basically we can / by log(N, 1<base<2, probably ~1.1)

0.010348171289556268

In [139]:
#So anyways getting back tothe key idea:
#We are predicting a series of decisions, i.e. some general hyperparameters, they can be whatever
#So basically any question we have, such as number of layers, number of hidden units, bottle neck units,
#controller layers, etc, can be put in there

In [140]:
#Applying it back to architecture search takes some tinkering
#I.e. at the end of the day we need to structure those decisions such that they can be used to make
#an architecture, such as what I have right now

In [141]:
#So I cant really predict a whole series of decisions and how they wire together, 
#unless I present the system with all of the choices, which will be a combinatorial explosion

In [142]:
#Rather for example we can design a system that fits into an input and an output
#and have the system design anything using that.

In [143]:
#So for example we can choose if we want to coadapt the components (probably best)
#We will generate an architecture matching each of those specifications (input and output matching)
#and then we will judge it how we do right now

In [144]:
#So basically we would have a decision making controller, which is specialized to the domain we care about?
#In this case it's all the domain of designing architectures.

In [145]:
#So again: The main things are:...
#1. The system makes generic decisions, it can be anything as long as we are getting a reward signal.
#2. Any decision that may be biased, such as the number of sims where more is almost always better, we should
#divide by some function of the max number of choices. So for example, maybe we can divide by between
#log(base=1.1-1.9), depending on how we want to balance between improvement per unit
#. Unless we make a full program synthesizer, we dont have any real hope of giving the domain to the architecture
#builder something like alpha zero. It requires loops, storing variables, etc, and would be nearly impossible
#to replicate directly. We can maybe emulate it using a neural network however, such as viewing a state,
#generating a "UCT" policy, etc, probably for networks it may be better to work with full states rather than
#policies, or for example we can do additive attention policies to shift (if applicable)

In [146]:
#In theory we could make a neural network approximation of whatever we want, we just need to ensure
#we have a rich enough set of options

In [147]:
#So anyways yeah the idea is that right now I need to increase the expressivity
#such as non linear path through the nodes, being able to choose whether to stop, etc

In [148]:
#How would that work?

In [149]:
#Basically, we want an upper bound on what we allow the program to do.
#Like lets say, if the parameter count is > than 10million, it is automatically considered a failure

In [150]:
#And the idea is that we give the program that upper bound, and then let it do anything within that

In [151]:
#So it should be basically impossible for the system to overload memory

In [152]:
#So lets say for example we calculate, max_layers*max_filters*flattened_input_size*max_batch_size=
#num parameters (or something)
#and we dont want it to exceed what we expect would be bad

In [164]:
max_batch_size = 512
flattened_input_size = 3*32*32
max_filters = 32
max_layers=10

#well lets leave one fixed for now
#lets say, we have a prior that we want 10 nodes
#nodes / variables, etc
#well basically we have a tradeoff. Do we want more filters or more layers
#seems like what you could do with more nodes is a heck of a lot more interesting
#so lets assume that the max filters is 256
num_params = max_layers*max_filters*flattened_input_size\
# *max_batch_size
#target = lets say (because I have a cpu)
#1 million parameters
# 1e6 = max_layers*max_filters*flattened_input_size


1000000/(max_filters*flattened_input_size)
#So for that number of filters for 1m params the max would be 1 layer
#Well this is useful

10.172526041666666

In [165]:
#So anyways we need to think about expressiveness
#Again we come back to having things be reachable
#Is LSTM reachable from the ENAS setup?

#They choose an input node, and an activation, and go through each of the nodes, then average the loose ends

In [ ]:
ft = σ(Uf@ht−1 + Wf@xt + bf)
ht = ft * ht−1 + (1 − ft) * tanh(Uh@ht−1 + Wh@xt + bh)


In [166]:
#So for example lets say we have nodes Uf, Wf

#.... Well this is going to get complicated
#Do I think it's reachable?
#You are choosing the output of a previous node

In [167]:
#So I think we need to store things as variables
#and really I think we want to be able to store an unlimited number of variables up to a certain point
#so for example, we choose dynamically, 

In [168]:
#So lets see, lets just try to make an LSTM using this

#So lets say we have a bunch of nodes
#Uf, ht-1, Wf, xt, Wh, Uh

#Basically we want to pick two nodes, and send the result somewhere else
#The issue is that we only keep that set of nodes, we 

#Basically the issue here is that we dont want to have to overwrite old nodes to hold intermediate representations
#it may be possible to do that, but its more complicated and has more potentials for failures, i.e. you need
#a very specific order to get the right function 

In [169]:
#So I think it's probably better to allow to dynamically increase the number of nodes you store

In [170]:
#So for example, lets say we determine a maximum number of parameters, and if we exceed that we early quit
#And then we need a maximum number of variables

In [171]:
#Or in fact, I dont think the memory concern is that much, so we could probably do unlimited variables,
#but a maximum number of decisions, i.e. after 500 steps we exit with what we have

In [172]:
#So anyways, lets assume w have a dynamic system
#Create Uf (could be a complicated conv), create Wf, create Wh, create Uh, etc
#for ht-1, we need to somehow store past results, so maybe we can specifically allow the net
#to ...

In [173]:
#so basically it would be that we need to have arbitrary symbol manipulation
#so we need to be able to have the flow of the net follow that
#so for example, we create our nodes, 

In [174]:
#So I guess the algorithm would be two steps, or its all going to be a sequential set of decisions, 
#but we will then use them in a certain way
#so lets see...

In [175]:
#Right now we go layer by layer and design all of the hyperparameters for those
#what we want to do is declare our variables, then choose how to wire them together

In [176]:
#So lets see. We want to make a dynamic system
#For that, we need to probably use what pointer nets did
#We want to be presented with a set of options

In [177]:
#So for example, we would do a top level call for a function which is "Make a conv layer"

In [178]:
#So basically the top level function would decide, quit, or make a new layer (node)
#so for make a new layer, it would basically be what we have now
#we go through each of the attributes, and we choose values that we believe are exceptable (basically
#2^n to 2^(n+k), choosing lower and upper bounds on what we believe are reasonable choices
#and at the end we have the full specification of a conv layer, bound to var0, and we can repeat that
#and make however many vars we want, up to a certain max layers

#when we quit we proceed to the graph flow

In [179]:
#So now we have all of these layers, and we need to set up the flow of the data through them

In [180]:
#We have one input that will always come in, i.e. the input to the program, and we have one output
#that will always go out that needs to be a certain size.
#and we will have the simplest stem and output to make things fit, if need be.

In [181]:
#So anyways, basically every choice is going to be pick input1, input2, and a target, as well as a function
#so basically for example lets go back to the forget gate LSTM example

In [ ]:
ft = σ(Uf@ht−1 + Wf@xt + bf)
ht = ft * ht−1 + (1 − ft) * tanh(Uh@ht−1 + Wh@xt + bh)


In [182]:
#We would pick input one, Uf, input2 ht-1, target -> new_var (all targets will be new vars, except maybe
#some special ones, idk), function = matrix multiply, and also we pick if there will be an activation

In [183]:
# ft = σ(((var0 + var1)=var2 + bf)=var3, activation =sigmoid)
# var3
# ht = var5=(var4=(ft * ht−1) + var9=(1 − ft)) * tanh(var6=(Uh@ht−1) + Wh@xt + bh)

In [184]:
#etc, etc
#So that basically convinces me that we can make whatever 2 input function using that system

In [185]:
#I should probably read more into program synthesis to get a better understanding, but I think from a 
#practical standpoint this is a good starting point

In [186]:
#Is there a way I could make it easier?
#I could maybe limit it so that I dont allow combining old vars with new vars, but I dont think that simplifies
#anything.

In [187]:
#having to specify the activation each time is a little annoying, but its not the end of the world

In [188]:
#We could maybe split thing into different parts, i.e. we have an activation part and...
#Doesnt stand out as being simpler

In [189]:
#Basically we allow an arbitrary number of operations, i.e. maybe 100, and we dont allow the net to
#overwrite vars, i.e. the new var will always come out.

In [190]:
#and at the end we need to finally choose which variable to make the final output

In [ ]:
#for simplicity we can either have the output of the net always 